# Download and Imports

In [ ]:
# Standard Libraries
import os
import random
import shutil
import copy
import warnings
import optuna
from pathlib import Path

# Numerical and Data Manipulation Libraries
import numpy as np

# Image Processing Libraries
import cv2
from PIL import Image

# Progress Bar
from tqdm import tqdm

# PyTorch Libraries
import torch

# TensorFlow and Keras Libraries
import tensorflow as tf
from tensorflow.keras import layers, backend as K
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, AdamW, SGD, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import (
    Layer, Input, Dense, Dropout, LayerNormalization, Flatten, Concatenate, multiply, BatchNormalization,
    Conv2D, GlobalMaxPooling2D, GlobalAveragePooling2D, Add, Multiply, MultiHeadAttention, Reshape, Permute, Activation, concatenate
)
from tensorflow.keras.applications import DenseNet201

# Machine Learning and Evaluation Libraries
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_curve, auc,
    accuracy_score, mean_squared_error
)
from imblearn.over_sampling import SMOTE

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

# Initialize Plotly for notebook
init_notebook_mode(connected=True)

# Remove Warnings
warnings.filterwarnings("ignore")

# Set Random Seed for Reproducibility
RANDOM_SEED = 123


In [ ]:
# Importing the path to dataset
data_dir = Path("/kaggle/input/breakhiss/HPI/100X")

In [ ]:
# Total image count inside the dataset
image_count = len(list(data_dir.glob('**/*.png')))
print(image_count)

In [ ]:
# Iterate over subfolders and count images in each subfolder
for subfolder in data_dir.glob('*'):
    if subfolder.is_dir():  # checks if it's a subfolder
        subfolder_name = subfolder.name
        image_count = len(list(subfolder.glob('*.png')))
        print(f"Folder: {subfolder_name}, Image Count: {image_count}")

In [ ]:
# Define the image size you want
IMG_SIZE = (224, 224)

NUM_CLASSES = 2

# List of subfolders (classes) in the dataset directory
classes = ['Benign', 'Malignant']

# Move the model to GPU if avilable
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!apt-get install tree
#clear_output()
# create new folders
!mkdir TRAIN TEST TRAIN/BENIGN TRAIN/MALIGNANT TEST/BENIGN TEST/MALIGNANT
!tree -d

# Train - Test Split

In [ ]:
IMG_PATH = '/kaggle/input/breakhiss/HPI/400X/'
TRAIN_PERCENT = 0.7
TEST_PERCENT = 0.3

# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if n < TRAIN_PERCENT * IMG_NUM:
                shutil.copy(img, 'TRAIN/' + CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'TEST/' + CLASS.upper() + '/' + FILE_NAME)

# Function to load images from directory

In [ ]:
def load_data(dir_path, img_size=(100,100)):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    img = cv2.resize(img, img_size)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels

In [ ]:
TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'

# use predefined function to load the image data into workspace
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)

In [ ]:
# Print the shapes
print("X_Train Shape:", X_train.shape)
print("y_train Shape:", y_train.shape)
# Check the data type of the preprocessed images
print(f"Data type of HP images (X_train): {X_train.dtype}")
print(f"Min pixel value: {np.min(X_train)}")
print(f"Max pixel value: {np.max(X_train)}")

In [ ]:
# Assuming you have a list of class labels, e.g., ['Benign', 'Malignant']
class_labels = set(y_test)  # Assuming 'y_resampled' contains your class labels

# Iterate through each class label and count images
for label in class_labels:
    image_count = sum(1 for item in y_test if item == label)
    print(f'Training Folder: {label}, Image Count: {image_count}')

In [ ]:
# Print the shapes
print("X_test Shape:", X_test.shape)
print("y_test Shape:", y_test.shape)
# Check the data type of the preprocessed images
print(f"Data type of HP images (X_test): {X_test.dtype}")
print(f"Min pixel value: {np.min(X_test)}")
print(f"Max pixel value: {np.max(X_test)}")

# Bar graph plot for visualization of distribution of dataset

In [ ]:
y = dict()
y[0] = []
y[1] = []
for set_name in (y_train, y_test):
    y[0].append(np.sum(set_name == 0))
    y[1].append(np.sum(set_name == 1))

trace0 = go.Bar(
    x=['Train Set', 'Test Set'],
    y=y[0],
    name='Benign',
    marker=dict(color='#33cc33'),
    opacity=0.7
)
trace1 = go.Bar(
    x=['Train Set', 'Test Set'],
    y=y[1],
    name='Malignant',
    marker=dict(color='#ff3300'),
    opacity=0.7
)
data = [trace0, trace1]
layout = go.Layout(
    title='Count of classes in each set',
    xaxis={'title': 'Set'},
    yaxis={'title': 'Count'}
)
fig = go.Figure(data, layout)
iplot(fig)

# Function to plot samples from each class

In [ ]:
def plot_samples(X, y, labels_dict, n=50):
    """
    Creates a gridplot for desired number of images (n) from the specified set
    """
    for index in range(len(labels_dict)):
        imgs = X[np.argwhere(y == index)][:n]
        j = 10
        i = int(n/j)

        plt.figure(figsize=(15,2))
        c = 1
        for img in imgs:
            plt.subplot(i,j,c)
            plt.imshow(img[0])

            plt.xticks([])
            plt.yticks([])
            c += 1
        plt.suptitle('Tumor: {}'.format(labels_dict[index]))
        plt.show()

In [ ]:
plot_samples(X_train, y_train, labels, 10)

# Demo augmentation with one sample image

In [ ]:
os.makedirs('preview', exist_ok=True)
x = X_train[0]  
x = x.reshape((1,) + x.shape) 

# set the paramters we want to change randomly
demo_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [ ]:
i = 0
for batch in demo_datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='aug_img', save_format='png'):
    i += 1
    if i > 20:
        break 

In [ ]:
plt.imshow(X_train[0])
plt.xticks([])
plt.yticks([])
plt.title('Original Image')
plt.show()

plt.figure(figsize=(15,6))
i = 1
for img in os.listdir('preview/'):
    img = cv2.imread('preview/' + img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(3,7,i)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    i += 1
    if i > 3*7:
        break
plt.suptitle('Augemented Images')
plt.show()

# Final Augmentation

In [ ]:
# Define benign and malignant images
benign_images = X_train[y_train == 0]
malignant_images = X_train[y_train == 1]

# Define augmentation factors
benign_augmentation_factor = 4
malignant_augmentation_factor = 2

In [ ]:
# Data Augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=40,                # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2,            # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,           # Randomly shift images vertically by 20% of the height
    shear_range=0.2,                  # Randomly apply shear transformations
    zoom_range=0.2,                   # Randomly zoom in on images
    horizontal_flip=True,             # Randomly flip images horizontally
    vertical_flip=True,               # Randomly flip images vertically
    preprocessing_function=lambda x: x + np.random.normal(0, 0.25, x.shape),  # Add Gaussian noise (mean=0, variance=0.25)
    fill_mode='nearest'               # Fill in any empty pixels after transformations with the nearest valid value
)

In [ ]:
# Function to apply augmentation, resize, and save images
def aug_images(image_list, label, augmentation_factor, label_name):
    all_images = []
    all_labels = []

    for image in image_list:
        all_images.append(image)
        all_labels.append(label)

    # Generate augmented images on-the-fly
    augmented_data = datagen.flow(np.array(all_images), np.array(all_labels), batch_size=len(all_images), shuffle=False)
    # Get augmented images and labels
    X_augmented, y_augmented = next(augmented_data)
    # Save augmented images
    augmented_path = f"Augmented/Augmented_{label_name}"
    os.makedirs(augmented_path, exist_ok=True)

    for i in range(len(all_images)):
        for j in range(augmentation_factor):
            index = i * augmentation_factor + j
            cv2.imwrite(os.path.join(augmented_path, f'aug_{index+1}.png'), X_augmented[index % len(X_augmented)])

In [ ]:
# Process Benign Images
aug_images(benign_images, 0, benign_augmentation_factor, 'Benign')

# Process Malignant Images
aug_images(malignant_images, 1, malignant_augmentation_factor, 'Malignant')

print("Augmentation completed successfully.")

In [ ]:
augmented_dir = Path("/kaggle/working/Augmented")

In [ ]:
# Iterate over subfolders and count images in each subfolder
for subfolder in augmented_dir.glob('*'):
    if subfolder.is_dir():  # checks if it's a subfolder
        subfolder_name = subfolder.name
        image_count = len(list(subfolder.glob('*.png')))
        print(f"Folder: {subfolder_name}, Image Count: {image_count}")

# Balance the augmented dataset using SMOTE(Synthetic Minority Oversampling Technique)

In [ ]:
Augumented_Img_Dir = "/kaggle/working/Augmented/"

# Initialize empty lists to store images and labels
X = []  # This will store image data
y = []  # This will store corresponding labels

In [ ]:
# All subfolders are appended to achieve a more balanced dataset
for subfolder in os.listdir(Augumented_Img_Dir):
    if os.path.isdir(os.path.join(Augumented_Img_Dir, subfolder)):
        label = subfolder
        subfolder_path = os.path.join(Augumented_Img_Dir, subfolder)

        # Iterate through images in the subfolder
        for image_file in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_file)

            try:
                # Load the image and append it to X
                image = Image.open(image_path)
                X.append(np.array(image))  # Convert image to numpy array

                # Append the label to y
                y.append(label)
            except Exception as e:
                print(f"Error loading image: {image_path}")
                print(f"Error message: {str(e)}")

print("Total images loaded:", len(X))
print("Total labels loaded:", len(y))

In [ ]:
# Convert X and y to numpy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
# Reshape each image to a flat 1D array
X_flat = [image.flatten() for image in X]
# Convert the list of flat arrays to a 2D NumPy array
X_flat = np.array(X_flat)

# Apply SMOTE to balance the dataset
smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=5)
X_resampled, y_resampled = smote.fit_resample(X_flat, y)

# Reshape the flattened images back to their original shape
X_balanced = X_resampled.reshape(-1, IMG_SIZE[0], IMG_SIZE[1], 3)

In [ ]:
# Assuming you have a list of class labels, e.g., ['Benign', 'Malignant']
class_labels = set(y_resampled)  # Assuming 'y_resampled' contains your class labels

# Iterate through each class label and count images
for label in class_labels:
    image_count = sum(1 for item in y_resampled if item == label)
    print(f'Training Folder: {label}, Image Count: {image_count}')

In [ ]:
# Assuming X_balanced contains images after balancing
# Plot a few balanced images to check color correctness and data type
num_images_to_plot = 5
for i in range(num_images_to_plot):
    plt.subplot(1, num_images_to_plot, i + 1)
    plt.imshow(X_balanced[i])  # Display the image
    plt.title(f"Image {i + 1}")
    plt.axis("off")
plt.show()

# Check the data type of the balanced images
print(f"Data type of balanced images (X_balanced): {X_balanced.dtype}")
print(f"Min pixel value: {np.min(X_balanced)}")
print(f"Max pixel value: {np.max(X_balanced)}")

# Save the images after balancing

In [ ]:
def save_img(images, labels, classes, save_dir):
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    for idx, (img, label) in enumerate(zip(images, labels)):
        class_dir = save_dir / classes[label]
        class_dir.mkdir(parents=True, exist_ok=True)
        img_path = class_dir / f"image_{idx + 1}.png"
        cv2.imwrite(str(img_path), img)
    print(f"Images saved to {save_dir}")

In [ ]:
aug_classes = ['Augmented_Benign', 'Augmented_Malignant']

# Map class names to numeric labels
label_mapping = {class_name: idx for idx, class_name in enumerate(aug_classes)}

# Convert y_resampled to numeric labels using the mapping
y_resampled_numeric = [label_mapping[label] for label in y_resampled]

# Save the Balanced images
save_img(X_balanced, y_resampled_numeric, aug_classes, "/kaggle/working/Balanced_HP_Images")

In [ ]:
# Check the shape of balanced dataset
print("X_balanced Shape:", X_balanced.shape)
print("y_resampled_numeric Shape:", y_resampled.shape)

# Load the augmented-balaced dataset for training

In [ ]:
TRAINING_DIR = "/kaggle/working/Balanced_HP_Images/"

# use predefined function to load the image data into workspace
X_training, y_training, labels = load_data(TRAINING_DIR, IMG_SIZE)

In [ ]:
plot_samples(X_training, y_training, labels, 10)

In [ ]:
# Check the shapes of the training  sets
print("X_training shape:", X_training.shape)
print("y_training shape:", y_training.shape)

In [ ]:
# Check the data type of the preprocessed_augmented images
print(f"Data type of HP images (X_training): {X_training.dtype}")
print(f"Min pixel value: {np.min(X_training)}")
print(f"Max pixel value: {np.max(X_training)}")

# Training and Validation split using StratifiedShuffleSplit

In [ ]:
# Define the percentages for training and validation sets
TRAIN_PERCENT = 0.7  # 70% for training
VAL_PERCENT = 0.3    # 30% for validation

# Initialize StratifiedShuffleSplit
stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=VAL_PERCENT, random_state=42)

# Perform stratified splitting
for train_index, val_index in stratified_splitter.split(X_training, y_training):
    X_Train, y_Train = X_training[train_index], y_training[train_index]
    X_val, y_val = X_training[val_index], y_training[val_index]

# Final Visualization of Class Distribution Across Datasets

In [ ]:
y = dict()
y[0] = []
y[1] = []
for set_name in (y_Train, y_val, y_test):
    y[0].append(np.sum(set_name == 0))
    y[1].append(np.sum(set_name == 1))

trace0 = go.Bar(
    x=['Train Set', 'Validation Set', 'Test Set'],
    y=y[0],
    name='Benign',
    marker=dict(color='#33cc33'),
    opacity=0.7
)
trace1 = go.Bar(
    x=['Train Set', 'Validation Set', 'Test Set'],
    y=y[1],
    name='Malignant',
    marker=dict(color='#ff3300'),
    opacity=0.7
)
data = [trace0, trace1]
layout = go.Layout(
    title='Count of classes in each set',
    xaxis={'title': 'Set'},
    yaxis={'title': 'Count'}
)
fig = go.Figure(data, layout)
iplot(fig)


In [ ]:
# Check the shapes of the training and validation sets
print("X_Train shape:", X_Train.shape)
print("y_Train shape:", y_Train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

In [ ]:
# Check the data type of the X_Train images
print(f"Data type of HP images (X_Train): {X_Train.dtype}")
print(f"Min pixel value: {np.min(X_Train)}")
print(f"Max pixel value: {np.max(X_Train)}")

# Save the images to training and validation directory

In [ ]:
# Create the directories if they don't exist
os.makedirs('Training_Dir', exist_ok=True)
os.makedirs('Validation_Dir', exist_ok=True)

# Use predefined function to load the image data into workspace
# Load resized images as np.arrays to the workspace
def save_images(directory, X, y, img_size=(100, 100)):
    """
    Save resized images as np.arrays to the specified directory
    """
    for i in tqdm(range(len(X))):
        label = labels[y[i]]
        img_name = f"{label}_{i}.png"
        img_path = os.path.join(directory, label, img_name)
        os.makedirs(os.path.dirname(img_path), exist_ok=True)
        img = cv2.resize(X[i], img_size)
        cv2.imwrite(img_path, img)

# Save training images
save_images('Training_Dir', X_Train, y_Train, IMG_SIZE)

# Save validation images
save_images('Validation_Dir', X_val, y_val, IMG_SIZE)

print("Images saved successfully")

# Defining ImageDataGenerator for Training, Validation, and Testing

In [ ]:
# Define directories for training, validation, and test data
TRAINING_DIR = 'Training_Dir/'
VALIDATION_DIR = 'Validation_Dir/'
TEST_DIR = 'TEST/'

# Create ImageDataGenerator for data augmentation and preprocessing for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,                  # Rescale pixel values to [0, 1] range
    rotation_range=20,               # Rotate images by up to 20 degrees
    width_shift_range=0.1,           # Shift images horizontally by up to 10%
    height_shift_range=0.1,          # Shift images vertically by up to 10%
    shear_range=0.15,                # Apply shear transformation
    zoom_range=0.15,                 # Apply zoom transformation
    horizontal_flip=True,            # Randomly flip images horizontally
    vertical_flip=False,             # Do not randomly flip images vertically
    fill_mode='nearest'              # Fill in missing pixels after transformation
)

# Create ImageDataGenerator for validation and test data preprocessing (no augmentation, only rescaling)
test_datagen = ImageDataGenerator(
    rescale=1./255  # Rescale pixel values to [0, 1] range
)

# Create training data generator
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,                    # Directory containing training images
    color_mode='rgb',                # Use RGB images
    target_size=IMG_SIZE,            # Resize images to target size
    batch_size=16,                   # Number of images to return in each batch
    class_mode='categorical',        # Return one-hot encoded labels
    shuffle=True,                    # Shuffle images randomly
    seed=RANDOM_SEED                 # Set random seed for reproducibility
)

# Create validation data generator
validation_generator = test_datagen.flow_from_directory(
    VALIDATION_DIR,                  # Directory containing validation images
    color_mode='rgb',                # Use RGB images
    target_size=IMG_SIZE,            # Resize images to target size
    batch_size=16,                   # Number of images to return in each batch
    class_mode='categorical',        # Return one-hot encoded labels
    shuffle=False,                   # Do not shuffle validation data
    seed=RANDOM_SEED                 # Set random seed for reproducibility
)

# Create test data generator
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,                        # Directory containing test images
    color_mode='rgb',                # Use RGB images
    target_size=IMG_SIZE,            # Resize images to target size
    batch_size=8,                   # Number of images to return in each batch
    class_mode='categorical',        # Return one-hot encoded labels
    shuffle=False,                   # Ensure consistent order of test data
    seed=RANDOM_SEED                 # Set random seed for reproducibility
)


# Propsed Model Design

In [ ]:
@register_keras_serializable()
class EnhancedAttentionMechanism(Layer):
    def __init__(self, filters, reduction=8, num_heads=3, dropout_rate=0.3, **kwargs):
        """
        Enhanced Attention Mechanism (EAM) with Multi-Head Self-Attention (MHSA).
        
        Parameters:
        - filters: Number of filters in the input tensor.
        - reduction: Reduction ratio for channel attention.
        - num_heads: Number of attention heads for MHSA.
        - dropout_rate: Dropout rate for regularization.
        """
        super(EnhancedAttentionMechanism, self).__init__(**kwargs)
        self.filters = filters
        self.reduction = reduction
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        # Channel Attention components
        self.global_max_pool = GlobalMaxPooling2D()
        self.dense1 = Dense(filters // reduction, activation='relu', kernel_regularizer=l2(1e-3))
        self.dense2 = Dense(filters, activation='sigmoid', kernel_regularizer=l2(1e-3))

        # Spatial Attention components
        self.conv_spatial = Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')

        # Multi-Head Self-Attention (MHSA)
        self.mhsa = MultiHeadAttention(num_heads=num_heads, key_dim=filters)

        # Dropout for regularization
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs):
        # Channel Attention (Global Context)
        avg_pool = self.global_max_pool(inputs)
        channel_attention = self.dense1(avg_pool)
        channel_attention = self.dense2(channel_attention)
        channel_attention = Reshape((1, 1, self.filters))(channel_attention)
        channel_refined = Multiply()([inputs, channel_attention])

        # Spatial Attention (Local Features)
        spatial_attention = self.conv_spatial(channel_refined)
        spatial_refined = Multiply()([inputs, spatial_attention])

        # Multi-Head Self-Attention (Long-Range Dependencies)
        # Flatten spatial dimensions to allow MHSA processing
        batch_size, height, width, channels = inputs.shape
        flattened_inputs = Reshape((height * width, channels))(inputs)
        mhsa_output = self.mhsa(flattened_inputs, flattened_inputs)
        mhsa_output = Reshape((height, width, channels))(mhsa_output)

        # Combine Channel, Spatial, and MHSA Outputs
        combined = Add()([channel_refined, spatial_refined, mhsa_output])
        return self.dropout(combined)

    def get_config(self):
        config = super(EnhancedAttentionMechanism, self).get_config()
        config.update({
            "filters": self.filters,
            "reduction": self.reduction,
            "num_heads": self.num_heads,
            "dropout_rate": self.dropout_rate,
        })
        return config


In [ ]:
# Initialize the DenseNet model without the top fully connected layers, using pre-trained ImageNet weights
base_model = DenseNet201(include_top=False, weights='imagenet', input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3))

# Save the input and output tensors of the base model
base_in = base_model.input
base_out = base_model.output

# Apply layer normalization to the output of the base model
x = LayerNormalization(epsilon=1e-6)(base_out)

# Initialize a custom self-attention layer with specified parameters
eam_layer = EnhancedAttentionMechanism(filters=x.shape[-1])

# Apply the self-attention layer to the normalized output
eam_op = eam_layer(x)

# Add the backbone output and the self-attention output (residual connection)
y = Add()([base_out, eam_op])

# Apply another layer normalization to the self-attention output
x = LayerNormalization(epsilon=1e-6)(y)

# Apply a 1x1 convolution with filters, l2 regularizer and ReLU activation
x = Conv2D(filters=x.shape[-1], kernel_size=1, padding='same', activation='relu')(x)

# Add the convolution output and the previous addition output (residual connection)
x = Add()([x, y])

# Apply global average pooling to reduce each feature map to a single value
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with softmax activation for classification
out = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the final model from the input of the base model to the output layer
model = Model(base_in, out, name="DREAMNet")
    
# Print the summary of the model
model.summary()


# Define callbacks

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_model.keras', 
    monitor='val_accuracy', 
    save_best_only=True, 
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True, 
    verbose=1
)


# Two stage model training

In [ ]:
# Set all layers in the model to be non-trainable (freeze the model)
for layer in base_model.layers:
    layer.trainable = False
    
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.005, decay=1e-6),
    metrics=['accuracy']
)

# Fit the model with callbacks
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=2,
    callbacks=[reduce_lr, checkpoint, early_stopping],
    verbose=1
)


In [ ]:
# Ensure that all layers in the model (including custom layers) are trainable
for layer in base_model.layers:
    layer.trainable = True
    
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=1e-4, decay=1e-5),
    metrics=['accuracy']
)

# Fit the model with callbacks
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[reduce_lr, checkpoint, early_stopping],
    verbose=1
)


# Graph plots

In [ ]:
# Plot training and validation loss and accuracy curves
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Load and evaluation of the model with different metrics

In [ ]:
# Step 1: Load the best model
best_model = load_model('best_model.keras', custom_objects={'EnhancedAttentionMechanism': EnhancedAttentionMechanism})

In [ ]:
# Step 2: Evaluate the model on test data
test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


In [ ]:
# Step 3: Generate predictions for the test set
predictions = best_model.predict(test_generator)
true_classes = test_generator.classes  # True labels
class_labels = list(test_generator.class_indices.keys())  # Class names

# Convert predictions to binary class labels
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
# Step 4: Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Step 5: Classification Report
print("Classification Report:\n", classification_report(true_classes, predicted_classes, target_names=class_labels))

In [ ]:
# Step 6: AUC and ROC Curve
# Get probability scores for the positive class (assuming class 1 is the positive class)
positive_class_probs = predictions[:, 1]
fpr, tpr, thresholds = roc_curve(true_classes, positive_class_probs)
roc_auc = auc(fpr, tpr)
print(f"AUC Score: {roc_auc:.4f}")

# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Step 7: Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(true_classes, predicted_classes))
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")

# Happy Coding!